# Lesson 06: OpenCV

OpenCV is a library that contains algorithms and functions that are related to and perform image processing and computer vision tasks.

In [1]:
import cv2

# Opening Images

In [8]:
img = cv2.imread('../images/test-image.png')
print(img.shape)

(512, 768, 3)


In [9]:
cv2.imshow('img', img)

# press 'q' to close image or else it will not be properly closed.
cv2.waitKey(0)
cv2.destroyAllWindows()

# Changing Color Space

In [10]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [11]:
cv2.imwrite('gray.jpg', gray)

True

In [12]:
cv2.imshow('img', gray)

# press 'q' to close image or else it will not be properly closed.
cv2.waitKey(0)
cv2.destroyAllWindows()

# Resizing

height, width = img.shape[:-1]
big_image = cv2.resize(img, (2*height, 2*width))
cv2.imshow('img', big_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Cropping

In [ ]:
img_crop = img[0:500, 300:500]
cv2.imshow('img', img_crop)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Thresholding

# Filters

## Gaussian Blur

# Edge Detection

## Canny Edge Detector

# Feature Detection

## SIFT

# Opening Videos

# Converting to Grayscale